In [3]:
# importeren packages
import pandas as pd
import numpy as np
#conda install -c anaconda py-xgboost #installeer XGBoost
import xgboost
import pickle
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
#conda install -c anaconda py-xgboost #installeer XGBoost
import xgboost
#from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score

In [4]:
# setten source path: later moet alle brondata naar datalake
source_path ="P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"

In [5]:
# inladen bestand met alle buurten en onafhankelijke variabelen als all
buurtendata = pd.read_csv(source_path + 'buurten.csv', encoding='utf-16')
buurtendata.rename(columns={"RLBBEV18":"RLBBEV",'KL18': "KL", 
               'RLBRMTR18': 'RLBRMTR', 'RLBWON18':'RLBWON' , 'RLBBEV18':'RLBBEV', 'RLBVRZ18':'RLBVRZ', 'RLBVEI18': 'RLBVEI', 'RLBFYS18': 'RLBFYS',}, inplace=True) 

In [6]:
# inladen bestand met scores als sc
sc = pd.read_excel(source_path + 'buurten voor experts 2e ronde.xlsx', sheet_name='buurten')
# alleen selecteren kolommen voor scores en oordelen
sc = sc[['buurtcode','EE','TK']]

In [5]:
# merge op buurtcode (die van 2019)
#######
df = pd.merge(buurtendata,sc, on='buurtcode', how = 'left')

In [41]:
df.shape

(13632, 115)

In [42]:
#######
feature_cols = ['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww',
       'woxdelta5jrGem', 'wox', 'RLBBEV',
       'RLBVRZ', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10',
       'wv2000-2010', 'AantalBios5']

       
X = df[feature_cols]
y = df['EE']

Xs = df[df['EE'].notnull()].drop('EE', axis = 1)
ys = df[df['EE'].notnull()]['EE']
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.25, random_state=2)

In [43]:
X.columns

Index(['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww', 'woxdelta5jrGem', 'wox',
       'RLBBEV', 'RLBVRZ', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10', 'wv2000-2010',
       'AantalBios5'],
      dtype='object')

In [44]:
X.head()

,AantalCafe3,woxdelta5jr,woxm2,woxww,woxdelta5jrGem,wox,RLBBEV,RLBVRZ,AantalLevensm3,woxm2WG,aand1945-1970,woxdelta5jrWGabs,woxdelta1jrGem,AantalPodiumk10,wv2000-2010,AantalBios5
0,5.0,NaN,NaN,NaN,NaN,NaN,0.033864,-0.100124,14.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0
1,5.6,NaN,NaN,NaN,NaN,NaN,0.048271,-0.124013,14.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0
2,5.0,NaN,NaN,NaN,NaN,NaN,-0.005198,-0.114426,14.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0
3,4.6,NaN,NaN,NaN,NaN,NaN,0.105121,-0.139309,12.5,NaN,NaN,NaN,NaN,1.0,NaN,0.0
4,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7,NaN,NaN,NaN,NaN,1.0,NaN,0.0


# EE

In [45]:
# get dataset
feature_cols = ['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww',
       'woxdelta5jrGem', 'wox', 'RLBBEV',
       'RLBVRZ', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10',
       'wv2000-2010', 'AantalBios5']      
X = df[feature_cols]
y = df['EE']

Xs = df[df['EE'].notnull()].drop('EE', axis = 1)
ys = df[df['EE'].notnull()]['EE']
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.25, random_state=2)

# remove testset
X_min_test = X.append(X_test)
X_min_test = X_min_test.loc[X_min_test.index.drop_duplicates(keep=False)]
y_min_test = y.append(y_test)
y_min_test = y_min_test.loc[y_min_test.index.drop_duplicates(keep=False)]
print(f"Removed test: { len(X)-len(X_test)==len(X_min_test)}")

# normalize dataset
scaler = RobustScaler()

# transform our full-dataset
X_min_test = scaler.fit_transform(X_min_test[feature_cols])
X_min_test = pd.DataFrame(X_min_test)
X_min_test.columns = feature_cols

# # select train items based on what's filled and not in test
X_train = X_min_test[y_min_test.notnull().values]
y_train = y_min_test[y_min_test.notnull().values]

# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X_train, y_train)

# get full dataset and normalize full dataset
X_full_transformed = scaler.fit_transform(X[feature_cols])
X_full_transformed = pd.DataFrame(X_full_transformed)
X_full_transformed.columns = feature_cols
X_full_transformed = X_full_transformed[df['EE'].notnull()] # only take labeled data
y = df[df['EE'].notnull()]['EE']
tempx, X_test, tempy, y_test = train_test_split(X_full_transformed, y, test_size=0.25, random_state=2)


# predict full dataset
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)

Removed test: True
0.9960474308300395 
 0.9294117647058824 
 [[34  3  0]
 [ 2 27  0]
 [ 1  0 18]]


In [46]:
fi = pd.DataFrame(model.feature_importances_,X_train.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
woxm2,0.215877,woxm2
woxww,0.181061,woxww
woxm2WG,0.104963,woxm2WG
woxdelta5jr,0.082669,woxdelta5jr
wox,0.062444,wox
AantalPodiumk10,0.051171,AantalPodiumk10
wv2000-2010,0.041772,wv2000-2010
AantalBios5,0.041467,AantalBios5
woxdelta5jrWGabs,0.038685,woxdelta5jrWGabs
RLBBEV,0.036130,RLBBEV


In [47]:
# sla model op
file_name = "final_models_normalised/final_model_EE_wox.pkl"
pickle.dump(model, open(file_name, "wb"))

## Thomas model

In [48]:
# get dataset
feature_cols = ['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww',
       'woxdelta5jrGem', 'wox', 'RLBBEV',
       'RLBVRZ', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10',
       'wv2000-2010', 'AantalBios5']      
X = df[feature_cols]
y = df['TK']

Xs = df[df['TK'].notnull()].drop('TK', axis = 1)
ys = df[df['TK'].notnull()]['TK']
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.25, random_state=2)

# remove testset
X_min_test = X.append(X_test)
X_min_test = X_min_test.loc[X_min_test.index.drop_duplicates(keep=False)]
y_min_test = y.append(y_test)
y_min_test = y_min_test.loc[y_min_test.index.drop_duplicates(keep=False)]
print(f"Removed test: { len(X)-len(X_test)==len(X_min_test)}")

# normalize dataset
scaler = RobustScaler()

# transform our full-dataset
X_min_test = scaler.fit_transform(X_min_test[feature_cols])
X_min_test = pd.DataFrame(X_min_test)
X_min_test.columns = feature_cols

# # select train items based on what's filled and not in test
X_train = X_min_test[y_min_test.notnull().values]
y_train = y_min_test[y_min_test.notnull().values]

# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X_train, y_train)

# get full dataset and normalize full dataset
X_full_transformed = scaler.fit_transform(X[feature_cols])
X_full_transformed = pd.DataFrame(X_full_transformed)
X_full_transformed.columns = feature_cols
X_full_transformed = X_full_transformed[df['TK'].notnull()] # only take labeled data
y = df[df['TK'].notnull()]['TK']
tempx, X_test, tempy, y_test = train_test_split(X_full_transformed, y, test_size=0.25, random_state=2)


# predict full dataset
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)

Removed test: True
1.0 
 0.8823529411764706 
 [[31  2  4]
 [ 2 14  1]
 [ 1  0 30]]


In [49]:
fi = pd.DataFrame(model.feature_importances_,X_train.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
woxdelta1jrGem,0.097575,woxdelta1jrGem
RLBVRZ,0.097332,RLBVRZ
wox,0.093619,wox
woxdelta5jr,0.087776,woxdelta5jr
woxdelta5jrGem,0.081800,woxdelta5jrGem
woxm2WG,0.070157,woxm2WG
woxm2,0.066518,woxm2
woxww,0.066029,woxww
AantalBios5,0.056493,AantalBios5
AantalCafe3,0.047549,AantalCafe3


In [50]:
# sla model op
file_name = "final_models_normalised/final_model_TK_wox.pkl"
pickle.dump(model, open(file_name, "wb"))

## NO WOX Thomas

In [51]:
# feature_cols_nw = ['inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL', 
#                'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBVEI', 'RLBFYS', 
#                'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
#                'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
#                'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
#                'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen'] 

In [8]:
# get dataset
feature_cols = ['inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL', 
               'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBVEI', 'RLBFYS', 
               'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
               'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
               'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
               'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen']     
X = df[feature_cols]
y = df['TK']

Xs = df[df['TK'].notnull()].drop('TK', axis = 1)
ys = df[df['TK'].notnull()]['TK']
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.25, random_state=2)

# remove testset
X_min_test = X.append(X_test)
X_min_test = X_min_test.loc[X_min_test.index.drop_duplicates(keep=False)]
y_min_test = y.append(y_test)
y_min_test = y_min_test.loc[y_min_test.index.drop_duplicates(keep=False)]
print(f"Removed test: { len(X)-len(X_test)==len(X_min_test)}")

# normalize dataset
scaler = RobustScaler()

# transform our full-dataset
X_min_test = scaler.fit_transform(X_min_test[feature_cols])
X_min_test = pd.DataFrame(X_min_test)
X_min_test.columns = feature_cols

# # select train items based on what's filled and not in test
X_train = X_min_test[y_min_test.notnull().values]
y_train = y_min_test[y_min_test.notnull().values]

# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X_train, y_train)

# get full dataset and normalize full dataset
X_full_transformed = scaler.fit_transform(X[feature_cols])
X_full_transformed = pd.DataFrame(X_full_transformed)
X_full_transformed.columns = feature_cols
X_full_transformed = X_full_transformed[df['TK'].notnull()] # only take labeled data
y = df[df['TK'].notnull()]['TK']
tempx, X_test, tempy, y_test = train_test_split(X_full_transformed, y, test_size=0.25, random_state=2)


# predict full dataset
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)

Removed test: True
1.0 
 0.8705882352941177 
 [[32  4  1]
 [ 2 11  4]
 [ 0  0 31]]


In [9]:
fi = pd.DataFrame(model.feature_importances_,X_train.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
RLBVRZ,0.094495,RLBVRZ
AantalLevensm3,0.072214,AantalLevensm3
AantalBios5,0.071250,AantalBios5
fiets,0.054135,fiets
auto60,0.052623,auto60
AantalPodiumk10,0.051230,AantalPodiumk10
ov60,0.040819,ov60
AantalCafe3,0.039431,AantalCafe3
AantalMusea10,0.034068,AantalMusea10
hh30inkHoog,0.031017,hh30inkHoog


In [54]:
file_name = "final_models_normalised/final_model_TK_nowox.pkl"
pickle.dump(model, open(file_name, "wb"))

## NO WOX Erwin

In [10]:
# get dataset
# get dataset
feature_cols = ['inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL', 
               'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBVEI', 'RLBFYS', 
               'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
               'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
               'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
               'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen']    
X = df[feature_cols]
y = df['EE']

Xs = df[df['EE'].notnull()].drop('EE', axis = 1)
ys = df[df['EE'].notnull()]['EE']
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.25, random_state=2)

# remove testset
X_min_test = X.append(X_test)
X_min_test = X_min_test.loc[X_min_test.index.drop_duplicates(keep=False)]
y_min_test = y.append(y_test)
y_min_test = y_min_test.loc[y_min_test.index.drop_duplicates(keep=False)]
print(f"Removed test: { len(X)-len(X_test)==len(X_min_test)}")

# normalize dataset
scaler = RobustScaler()

# transform our full-dataset
X_min_test = scaler.fit_transform(X_min_test[feature_cols])
X_min_test = pd.DataFrame(X_min_test)
X_min_test.columns = feature_cols

# # select train items based on what's filled and not in test
X_train = X_min_test[y_min_test.notnull().values]
y_train = y_min_test[y_min_test.notnull().values]

# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X_train, y_train)

# get full dataset and normalize full dataset
X_full_transformed = scaler.fit_transform(X[feature_cols])
X_full_transformed = pd.DataFrame(X_full_transformed)
X_full_transformed.columns = feature_cols
X_full_transformed = X_full_transformed[df['EE'].notnull()] # only take labeled data
y = df[df['EE'].notnull()]['EE']
tempx, X_test, tempy, y_test = train_test_split(X_full_transformed, y, test_size=0.25, random_state=2)


# predict full dataset
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
print(train_accuracy, '\n',test_accuracy,'\n', cm)


Removed test: True
1.0 
 0.8705882352941177 
 [[32  3  2]
 [ 2 25  2]
 [ 2  0 17]]


In [11]:
fi = pd.DataFrame(model.feature_importances_,X_train.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
AantalPodiumk10,0.108728,AantalPodiumk10
AantalMusea10,0.092169,AantalMusea10
fiets,0.050634,fiets
aandeelMH19,0.042041,aandeelMH19
banen,0.034482,banen
RLBRMTR,0.034147,RLBRMTR
RLBBEV,0.033882,RLBBEV
AfstandZwembad,0.031308,AfstandZwembad
hh30inkHoog,0.029784,hh30inkHoog
inw,0.029764,inw


In [57]:
file_name = "final_models_normalised/final_model_EE_nowox.pkl"
pickle.dump(model, open(file_name, "wb"))

# Voorspellingen gebruiken/ testen

In [8]:
feature_cols = ['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww',
       'woxdelta5jrGem', 'wox', 'RLBBEV',
       'RLBVRZ', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10',
       'wv2000-2010', 'AantalBios5']    

feature_cols_nw =  ['inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL', 
               'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBVEI', 'RLBFYS', 
               'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
               'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
               'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
               'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen']  
scaler = RobustScaler()

In [9]:
# de predictie dataframes maken:
# X_pred_w = buurtendata[feature_cols]
# X_pred_w = pd.DataFrame(X_pred_w)
# X_pred_w.columns = feature_cols

X_pred_w = buurtendata[feature_cols]
X_pred_w = scaler.fit_transform(X_pred_w[feature_cols])
X_pred_w = pd.DataFrame(X_pred_w)
X_pred_w.columns = feature_cols


# No wox:
# X_pred_nw = buurtendata[feature_cols_nw]
# X_pred_nw = pd.DataFrame(X_pred_nw)
# X_pred_nw.columns = feature_cols_nw

X_pred_nw = buurtendata[feature_cols_nw]
X_pred_nw = scaler.fit_transform(X_pred_nw[feature_cols_nw])
X_pred_nw = pd.DataFrame(X_pred_nw)
X_pred_nw.columns = feature_cols_nw

In [11]:
# laad model
model_2019_EE = pickle.load(open("../final_models_normalised/final_model_EE_wox.pkl", "rb"))
model_2019_TK = pickle.load(open("../final_models_normalised/final_model_TK_wox.pkl", "rb"))
model_2019_EE_nw = pickle.load(open("../final_models_normalised/final_model_EE_nowox.pkl", "rb"))
model_2019_TK_nw = pickle.load(open("../final_models_normalised/final_model_TK_nowox.pkl", "rb"))

In [12]:
predictions_EE = model_2019_EE.predict(X_pred_w)
predictions_TK = model_2019_TK.predict(X_pred_w)
predictions_EE_nw = model_2019_EE_nw.predict(X_pred_nw)
predictions_TK_nw = model_2019_TK_nw.predict(X_pred_nw)

In [13]:
buurtendata['voorspellingen_TK'] = predictions_TK
buurtendata['voorspellingen_EE'] = predictions_EE
buurtendata['voorspellingen_TK_nw'] = predictions_TK_nw
buurtendata['voorspellingen_EE_nw'] = predictions_EE_nw

In [14]:
len(buurtendata)

13632

In [15]:
df_final = pd.merge(buurtendata,sc,how='left', on='buurtcode')

In [16]:
df_final.shape

(13632, 119)

In [17]:
df_temp = df_final[df_final['EE'].notnull()]
print("accuracy score EE normal ", accuracy_score(df_temp['EE'], df_temp['voorspellingen_EE']))
print("accuracy score TK normal ", accuracy_score(df_temp['TK'], df_temp['voorspellingen_TK']))
print("accuracy score EE no wox ", accuracy_score(df_temp['EE'], df_temp['voorspellingen_EE_nw']))
print("accuracy score TK no wox ", accuracy_score(df_temp['TK'], df_temp['voorspellingen_TK_nw']))

accuracy score EE normal  0.9763313609467456
accuracy score TK normal  0.9674556213017751
accuracy score EE no wox  0.9674556213017751
accuracy score TK no wox  0.9674556213017751


In [18]:
df_final[['buurtcode', 'EE', 'TK', 'voorspellingen_EE', 'voorspellingen_TK','voorspellingen_EE_nw', 'voorspellingen_TK_nw']].head()

,buurtcode,EE,TK,voorspellingen_EE,voorspellingen_TK,voorspellingen_EE_nw,voorspellingen_TK_nw
0,30000,NaN,NaN,c,c,c,c
1,30001,NaN,NaN,c,c,b,c
2,30002,NaN,NaN,c,c,c,c
3,30007,NaN,NaN,c,c,b,c
4,30008,NaN,NaN,c,c,c,c


In [21]:
df_final['wox_info'] = df_final['wox'].notnull()


In [22]:
df_final['EE_Choice'] = df_final.apply(lambda x: x['voorspellingen_EE'] if x['wox_info'] else x['voorspellingen_EE_nw'], axis=1)
df_final['TK_Choice'] = df_final.apply(lambda x: x['voorspellingen_TK'] if x['wox_info'] else x['voorspellingen_TK_nw'], axis=1)

In [23]:
df_final[['buurtcode', 'EE', 'TK', 'voorspellingen_EE', 'voorspellingen_TK','voorspellingen_EE_nw', 'voorspellingen_TK_nw', 'wox_info', 'EE_Choice', 'TK_Choice']].to_csv('../predictions buurtscores/predictions_final_normalised.csv')

In [20]:
sum(df_final['wox'].notnull())

6737